### Aggregating and Combining pandas DataFrames

In [1]:
import pandas as pd
import numpy as np
import requests as rq
from sklearn.preprocessing import OneHotEncoder
from zipfile import ZipFile

c:\Users\Gmwende\anaconda3\envs\learn-env\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


### Learning Goals
- Use GroupBy objects to organize and aggregate data
- Create pivot tables from DataFrames
- Combine DataFrames by merging and appending

We'll work with the Austin Animal Center dataset and with data from King County's Department of Assessments (Seattle housing data).
### Austin Animal Center Data

In [2]:
##HTTPSConnectionPool(host='data.austintexas.gov', port=443): Max retries exceeded with url: /resource/9t4d-g238.json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1123)')))
## add verify=False to skip verifications
data = rq.get('https://data.austintexas.gov/resource/9t4d-g238.json',verify=False).text

c:\Users\Gmwende\anaconda3\envs\learn-env\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.austintexas.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [3]:
animals = pd.read_json(data)

In [4]:
animals.head()

,animal_id,name,datetime,monthyear,date_of_birth,outcome_type,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,outcome_subtype
0,A882831,*Hamilton,2023-07-01 18:12:00,2023-07-01T18:12:00.000,2023-03-25T00:00:00.000,Adoption,Cat,Neutered Male,3 months,Domestic Shorthair Mix,Black/White,NaN
1,A794011,Chunk,2019-05-08 18:20:00,2019-05-08T18:20:00.000,2017-05-02T00:00:00.000,Rto-Adopt,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White,NaN
2,A776359,Gizmo,2018-07-18 16:02:00,2018-07-18T16:02:00.000,2017-07-12T00:00:00.000,Adoption,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown,NaN
3,A821648,NaN,2020-08-16 11:38:00,2020-08-16T11:38:00.000,2019-08-16T00:00:00.000,Euthanasia,Other,Unknown,1 year,Raccoon,Gray,NaN
4,A720371,Moose,2016-02-13 17:59:00,2016-02-13T17:59:00.000,2015-10-08T00:00:00.000,Adoption,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff,NaN


### Aggregating over DataFrames: .groupby()
Those of you familiar with SQL have probably used the GROUP BY command. (And if you haven't, you'll see it very soon!) Pandas has this, too.

The .groupby() method is especially useful for aggregate functions applied to the data grouped in particular ways

In [5]:
animals.groupby('animal_type')

In [6]:
animals.columns

Index(['animal_id', 'name', 'datetime', 'monthyear', 'date_of_birth',
       'outcome_type', 'animal_type', 'sex_upon_outcome', 'age_upon_outcome',
       'breed', 'color', 'outcome_subtype'],
      dtype='object')

We can group by multiple columns, and also return a DataFrameGroupBy object

Notice the object type DataFrameGroupBy object.
### .groups and .get_group()

In [7]:
animals.groupby(['animal_type','outcome_type'])

In [8]:
animals.groupby('animal_type').groups #gives rows/indexes the type is in

{'Bird': [143, 257, 660], 'Cat': [0, 1, 5, 8, 9, 10, 11, 13, 14, 15, 18, 19, 21, 22, 23, 24, 25, 27, 30, 31, 33, 37, 39, 47, 50, 62, 65, 73, 76, 88, 89, 90, 92, 98, 102, 105, 108, 109, 113, 115, 116, 118, 120, 121, 123, 124, 128, 142, 145, 146, 147, 148, 149, 151, 153, 154, 159, 167, 173, 174, 176, 178, 179, 180, 183, 185, 186, 189, 190, 191, 194, 195, 196, 200, 203, 206, 207, 209, 212, 213, 218, 220, 227, 231, 236, 237, 239, 241, 242, 245, 248, 250, 252, 253, 254, 258, 263, 266, 267, 270, ...], 'Dog': [2, 4, 6, 7, 12, 16, 17, 20, 26, 29, 32, 34, 35, 36, 38, 41, 42, 43, 44, 45, 48, 49, 51, 52, 54, 55, 56, 57, 58, 60, 61, 63, 64, 66, 67, 68, 69, 70, 71, 74, 75, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 91, 95, 96, 97, 99, 101, 103, 104, 106, 107, 110, 111, 112, 114, 117, 119, 122, 125, 126, 127, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 144, 150, 152, 155, 156, 157, 158, 160, 161, 162, 163, 164, 165, 166, 168, 169, ...], 'Other': [3, 28, 40, 46, 53, 59, 72, 93, 94, 1

In [9]:
animals['animal_type'].value_counts() #testing code

Dog      570
Cat      378
Other     49
Bird       3
Name: animal_type, dtype: int64

In [10]:
animals.groupby('animal_type').get_group('Bird')

,animal_id,name,datetime,monthyear,date_of_birth,outcome_type,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,outcome_subtype
143,A878874,A878874,2023-04-22 10:44:00,2023-04-22T10:44:00.000,2022-04-18T00:00:00.000,Died,Bird,Unknown,1 year,Parakeet,Green,In Kennel
257,A720727,Rooster 11,2016-03-08 13:47:00,2016-03-08T13:47:00.000,2015-02-14T00:00:00.000,Adoption,Bird,Intact Male,1 year,Chicken Mix,Black/Red,NaN
660,A720734,Rooster 18,2016-03-08 15:07:00,2016-03-08T15:07:00.000,2015-02-14T00:00:00.000,Adoption,Bird,Intact Male,1 year,Chicken Mix,Black/Chocolate,NaN


### Multiindexing

In [11]:
animal_outcome = animals.groupby(['animal_type','outcome_type'])
animal_outcome.groups

{('Bird', 'Adoption'): [257, 660], ('Bird', 'Died'): [143], ('Cat', 'Adoption'): [0, 8, 9, 10, 11, 23, 24, 30, 31, 33, 39, 62, 73, 76, 89, 92, 105, 108, 109, 113, 115, 118, 123, 124, 128, 149, 151, 167, 173, 176, 179, 180, 185, 189, 190, 191, 194, 196, 200, 203, 206, 209, 220, 227, 231, 237, 239, 241, 248, 250, 252, 253, 254, 258, 263, 271, 281, 285, 288, 307, 312, 314, 322, 326, 333, 334, 335, 341, 343, 347, 348, 351, 354, 357, 360, 363, 367, 370, 373, 381, 383, 393, 398, 403, 405, 407, 408, 412, 414, 416, 431, 437, 452, 454, 463, 465, 468, 471, 476, 484, ...], ('Cat', 'Died'): [65, 364, 551, 590, 837, 862], ('Cat', 'Disposal'): [571, 644], ('Cat', 'Euthanasia'): [22, 27, 47, 147, 148, 212, 284, 353, 382, 423, 466, 475, 556, 557, 649, 708, 712, 997], ('Cat', 'Return to Owner'): [15, 98, 120, 186, 378, 418, 550, 653, 700, 706, 725, 929, 995], ('Cat', 'Rto-Adopt'): [1, 376, 507], ('Cat', 'Transfer'): [5, 13, 14, 18, 19, 21, 25, 37, 50, 88, 90, 102, 116, 121, 142, 145, 146, 153, 154, 159

In [12]:
animal_outcome.keys

['animal_type', 'outcome_type']

In [13]:
animal_outcome.get_group(('Cat','Adoption'))

,animal_id,name,datetime,monthyear,date_of_birth,outcome_type,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,outcome_subtype
0,A882831,*Hamilton,2023-07-01 18:12:00,2023-07-01T18:12:00.000,2023-03-25T00:00:00.000,Adoption,Cat,Neutered Male,3 months,Domestic Shorthair Mix,Black/White,NaN
8,A902098,NaN,2024-07-18 12:07:00,2024-07-18T12:07:00.000,2024-04-10T00:00:00.000,Adoption,Cat,Neutered Male,3 months,Domestic Shorthair,Blue Tabby,Foster
9,A860161,*Lalo,2022-07-19 15:53:00,2022-07-19T15:53:00.000,2022-05-04T00:00:00.000,Adoption,Cat,Neutered Male,2 months,Domestic Shorthair,Brown Tabby,NaN
10,A689724,*Donatello,2014-10-18 18:52:00,2014-10-18T18:52:00.000,2014-08-01T00:00:00.000,Adoption,Cat,Neutered Male,2 months,Domestic Shorthair Mix,Black,NaN
11,A680969,*Zeus,2014-08-05 16:59:00,2014-08-05T16:59:00.000,2014-06-03T00:00:00.000,Adoption,Cat,Neutered Male,2 months,Domestic Shorthair Mix,White/Orange Tabby,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
977,A850162,*Angelina Purrline,2022-01-25 16:35:00,2022-01-25T16:35:00.000,2021-01-19T00:00:00.000,Adoption,Cat,Spayed Female,1 year,Domestic Medium Hair,Brown Tabby/White,NaN
980,A729191,*Mr. Jones,2016-07-17 17:13:00,2016-07-17T17:13:00.000,2016-05-14T00:00:00.000,Adoption,Cat,Neutered Male,2 months,Domestic Shorthair Mix,Brown Tabby,NaN
989,A738723,*Celene,2016-12-09 09:20:00,2016-12-09T09:20:00.000,2016-10-11T00:00:00.000,Adoption,Cat,Spayed Female,1 month,Domestic Shorthair,Tortie,Foster
993,A727613,NaN,2016-06-30 17:15:00,2016-06-30T17:15:00.000,2016-03-23T00:00:00.000,Adoption,Cat,Neutered Male,3 months,Domestic Shorthair Mix,Torbie,NaN


In [14]:
animals['outcome_type'].value_counts()

Adoption           504
Transfer           263
Return to Owner    145
Euthanasia          56
Died                15
Rto-Adopt           13
Disposal             4
Name: outcome_type, dtype: int64

### Aggregating
Once again, as we will see in SQL, groupby objects are intended to be used with aggregation. In SQL, we will see that our queries that include GROUP BY require aggregation performed on columns.

We can use .sum(), .mean(), .count(), .max(), .min()

In [15]:
animals.groupby('animal_type').count()

,animal_id,name,datetime,monthyear,date_of_birth,outcome_type,sex_upon_outcome,age_upon_outcome,breed,color,outcome_subtype
animal_type,,,,,,,,,,,
Bird,3,3,3,3,3,3,3,3,3,3,1
Cat,378,235,378,378,378,378,378,378,378,378,229
Dog,570,492,570,570,570,570,570,570,570,570,190
Other,49,11,49,49,49,49,49,49,49,49,38


### Exercise
Use .groupby() to find the most recently born of each (main) animal type

In [16]:
animals.columns

Index(['animal_id', 'name', 'datetime', 'monthyear', 'date_of_birth',
       'outcome_type', 'animal_type', 'sex_upon_outcome', 'age_upon_outcome',
       'breed', 'color', 'outcome_subtype'],
      dtype='object')

In [17]:
animals.groupby('animal_type')['date_of_birth'].max()

animal_type
Bird     2022-04-18T00:00:00.000
Cat      2024-07-11T00:00:00.000
Dog      2024-04-19T00:00:00.000
Other    2023-10-28T00:00:00.000
Name: date_of_birth, dtype: object

### Pivoting a DataFrame
.pivot_table()
Those of you familiar with Excel have probably used Pivot Tables. Pandas has a similar functionality.

Grouping by two different columns can be very helpful.

In [18]:
animals.groupby(['outcome_type','sex_upon_outcome']).agg(len)

animal_id  name  datetime  monthyear  \
outcome_type    sex_upon_outcome                                         
Adoption        Intact Female            12    12        12         12   
                Intact Male              11    11        11         11   
                Neutered Male           258   258       258        258   
                Spayed Female           222   222       222        222   
                Unknown                   1     1         1          1   
Died            Intact Female             3     3         3          3   
                Intact Male               3     3         3          3   
                Neutered Male             1     1         1          1   
                Spayed Female             2     2         2          2   
                Unknown                   6     6         6          6   
Disposal        Intact Male               1     1         1          1   
                Unknown                   3     3         3          3   
Euthanasia      Intact Female             7     7         7          7   
                Intact Male               8     8         8          8   
                Neutered Male             4     4         4          4   
                Spayed Female             2     2         2          2   
                Unknown                  35    35        35         35   
Return to Owner Intact Female            21    21        21         21   
                Intact Male              34    34        34         34   
                Neutered Male            49    49        49         49   
                Spayed Female            41    41        41         41   
Rto-Adopt       Neutered Male             6     6         6          6   
                Spayed Female             7     7         7          7   
Transfer        Intact Female            82    82        82         82   
                Intact Male              79    79        79         79   
                Neutered Male            40    40        40         40   
                Spayed Female            35    35        35         35   
                Unknown                  27    27        27         27   

                                  date_of_birth  animal_type  \
outcome_type    sex_upon_outcome                               
Adoption        Intact Female                12           12   
                Intact Male                  11           11   
                Neutered Male               258          258   
                Spayed Female               222          222   
                Unknown                       1            1   
Died            Intact Female                 3            3   
                Intact Male                   3            3   
                Neutered Male                 1            1   
                Spayed Female                 2            2   
                Unknown                       6            6   
Disposal        Intact Male                   1            1   
                Unknown                       3            3   
Euthanasia      Intact Female                 7            7   
                Intact Male                   8            8   
                Neutered Male                 4            4   
                Spayed Female                 2            2   
                Unknown                      35           35   
Return to Owner Intact Female                21           21   
                Intact Male                  34           34   
                Neutered Male                49           49   
                Spayed Female                41           41   
Rto-Adopt       Neutered Male                 6            6   
                Spayed Female                 7            7   
Transfer        Intact Female                82           82   
                Intact Male                  79           79   
                Neutered Male                40           40   
                Spaye

But it has the unsavory side effect of creating a two-level index. This can be a good time to use .pivot_table().

There is also a .pivot().

In [19]:
df = pd.DataFrame({
    'sex':['male','male','male','male','male',"female", "female", "female", "female"],
    'num_puppies': ["one", "one", "one", "two", "two", "one", "one", "two", "two"],
    "breed": ["terrier", "retriever", "retriever", "terrier","terrier", "retriever", "terrier", "terrier","retriever"],
    "past_owners": [1, 2, 2, 3, 3, 4, 5, 6, 7],
    "family_members": [2, 4, 5, 5, 6, 6, 8, 9, 9]
})
df

,sex,num_puppies,breed,past_owners,family_members
0,male,one,terrier,1,2
1,male,one,retriever,2,4
2,male,one,retriever,2,5
3,male,two,terrier,3,5
4,male,two,terrier,3,6
5,female,one,retriever,4,6
6,female,one,terrier,5,8
7,female,two,terrier,6,9
8,female,two,retriever,7,9


In [20]:
# This first example aggregates values by taking the sum.
table = pd.pivot_table(df,values='past_owners',index=['sex','num_puppies'],
                       columns=['breed'],aggfunc=np.sum
                       )
table

breed               retriever  terrier
sex    num_puppies                    
female one                4.0      5.0
       two                7.0      6.0
male   one                4.0      1.0
       two                NaN      6.0

In [21]:
table.reset_index()

breed,sex,num_puppies,retriever,terrier
0,female,one,4.0,5.0
1,female,two,7.0,6.0
2,male,one,4.0,1.0
3,male,two,NaN,6.0


In [22]:
table.index

MultiIndex([('female', 'one'),
            ('female', 'two'),
            (  'male', 'one'),
            (  'male', 'two')],
           names=['sex', 'num_puppies'])

### Back to Austin animals

In [23]:
animals.columns

Index(['animal_id', 'name', 'datetime', 'monthyear', 'date_of_birth',
       'outcome_type', 'animal_type', 'sex_upon_outcome', 'age_upon_outcome',
       'breed', 'color', 'outcome_subtype'],
      dtype='object')

In [24]:
animals['outcome_type'].value_counts()

Adoption           504
Transfer           263
Return to Owner    145
Euthanasia          56
Died                15
Rto-Adopt           13
Disposal             4
Name: outcome_type, dtype: int64

In [25]:
animals['sex_upon_outcome'].value_counts()

Neutered Male    358
Spayed Female    309
Intact Male      136
Intact Female    125
Unknown           72
Name: sex_upon_outcome, dtype: int64

In [26]:
animals.pivot_table(index = 'outcome_type',columns='sex_upon_outcome',aggfunc=len)

age_upon_outcome                                          \
sex_upon_outcome    Intact Female Intact Male Neutered Male Spayed Female   
outcome_type                                                                
Adoption                     12.0        11.0         258.0         222.0   
Died                          3.0         3.0           1.0           2.0   
Disposal                      NaN         1.0           NaN           NaN   
Euthanasia                    7.0         8.0           4.0           2.0   
Return to Owner              21.0        34.0          49.0          41.0   
Rto-Adopt                     NaN         NaN           6.0           7.0   
Transfer                     82.0        79.0          40.0          35.0   

                             animal_id                            \
sex_upon_outcome Unknown Intact Female Intact Male Neutered Male   
outcome_type                                                       
Adoption             1.0          12.0        11.0         258.0   
Died                 6.0           3.0         3.0           1.0   
Disposal             3.0           NaN         1.0           NaN   
Euthanasia          35.0           7.0         8.0           4.0   
Return to Owner      NaN          21.0        34.0          49.0   
Rto-Adopt            NaN           NaN         NaN           6.0   
Transfer            27.0          82.0        79.0          40.0   

                                        ...          name              \
sex_upon_outcome Spayed Female Unknown  ... Intact Female Intact Male   
outcome_type                            ...                             
Adoption                 222.0     1.0  ...          12.0        11.0   
Died                       2.0     6.0  ...           3.0         3.0   
Disposal                   NaN     3.0  ...           NaN         1.0   
Euthanasia                 2.0    35.0  ...           7.0         8.0   
Return to Owner           41.0     NaN  ...          21.0        34.0   
Rto-Adopt                  7.0     NaN  ...           NaN         NaN   
Transfer                  35.0    27.0  ...          82.0        79.0   

                                                     outcome_subtype  \
sex_upon_outcome Neutered Male Spayed Female Unknown   Intact Female   
outcome_type                                                           
Adoption                 258.0         222.0     1.0            12.0   
Died                       1.0           2.0     6.0             3.0   
Disposal                   NaN           NaN     3.0             NaN   
Euthanasia                 4.0           2.0    35.0             7.0   
Return to Owner           49.0          41.0     NaN            21.0   
Rto-Adopt                  6.0           7.0     NaN             NaN   
Transfer                  40.0          35.0    27.0            82.0   

                                                                  
sex_upon_outcome Intact Male Neutered Male Spayed Female Unknown  
outcome_type                                                      
Adoption                11.0         258.0         222.0     1.0  
Died                     3.0           1.0           2.0     6.0  
Disposal                 1.0           NaN           NaN     3.0  
Euthanasia               8.0           4.0           2.0    35.0  
Return to Owner         34.0          49.0          41.0     NaN  
Rto-Adopt                NaN           6.0           7.0     NaN  
Transfer                79.0          40.0          35.0    27.0  

[7 rows x 50 columns]

### Exercise
Use .pivot_table() to add up the number of my tasks by category. Hint: Use sum() as your aggregating function.

In [27]:
tasks = pd.DataFrame({'category': ['house', 'house', 'school', 'school'],
                      'descr': ['kitchen', 'laundry', 'git', 'Python'],
                      'priority': [2, 3, 4, 1], 'num_tasks': [2, 1, 2, 3]})

tasks

,category,descr,priority,num_tasks
0,house,kitchen,2,2
1,house,laundry,3,1
2,school,git,4,2
3,school,Python,1,3


In [28]:
# table = pd.pivot_table(df,values='past_owners',index=['sex','num_puppies'],
#                        columns=['breed'],aggfunc=np.sum
#                        )
tasks.pivot_table(index='category',values='num_tasks',aggfunc=sum)

,num_tasks
category,
house,3
school,5


In [29]:
#columsn on task, what does the 123 columns look like
tasks.pivot_table(index='category',columns='num_tasks')

priority          
num_tasks        1    2    3
category                    
house          3.0  2.0  NaN
school         NaN  4.0  1.0

### Methods for Combining DataFrames: .join(), .merge(), pd.concat()
#### .join()

In [30]:
toy1 = pd.DataFrame([[63, 142], [33, 47]], columns=['age', 'HP'])
toy2 = pd.DataFrame([[63, 100], [33, 200]], columns=['age', 'MP'])

toy1

,age,HP
0,63,142
1,33,47


In [31]:
toy2

,age,MP
0,63,100
1,33,200


In [32]:
toy1.join(toy2,lsuffix=1,rsuffix='2')

,age1,HP,age2,MP
0,63,142,63,100
1,33,47,33,200


### If we don't want to keep both,we could set the overlapping column as the index in each DataFrame:

In [33]:
toy1.set_index('age').join(toy2.set_index('age'))

,HP,MP
age,,
63,142,100
33,47,200


### .merge()

In [34]:
toy1.merge(toy2)

,age,HP,MP
0,63,142,100
1,33,47,200


In [35]:
ds_chars = pd.read_csv('data/ds_chars.csv',index_col=0)
ds_chars

,name,HP,home_state
0,greg,200,WA
1,miles,200,WA
2,alan,170,TX
3,alison,300,DC
4,rachel,200,TX


In [36]:
states = pd.read_csv('data/states.csv',index_col=0)
states

,state,nickname,capital
0,WA,evergreen,Olympia
1,TX,alamo,Austin
2,DC,district,Washington
3,OH,buckeye,Columbus
4,OR,beaver,Salem


### The how Parameter
This parameter in both .join() and .merge() tells the compiler what sort of join to effect. We'll cover this in detail when we discuss SQL.

In [37]:
ds_chars.merge(states,
               left_on='home_state',
               right_on='state',
               how='inner')

,name,HP,home_state,state,nickname,capital
0,greg,200,WA,WA,evergreen,Olympia
1,miles,200,WA,WA,evergreen,Olympia
2,alan,170,TX,TX,alamo,Austin
3,rachel,200,TX,TX,alamo,Austin
4,alison,300,DC,DC,district,Washington


In [38]:
ds_chars.merge(states,
               left_on='home_state',
               right_on='state',
               how='outer')

,name,HP,home_state,state,nickname,capital
0,greg,200.0,WA,WA,evergreen,Olympia
1,miles,200.0,WA,WA,evergreen,Olympia
2,alan,170.0,TX,TX,alamo,Austin
3,rachel,200.0,TX,TX,alamo,Austin
4,alison,300.0,DC,DC,district,Washington
5,NaN,NaN,NaN,OH,buckeye,Columbus
6,NaN,NaN,NaN,OR,beaver,Salem


### pd.concat()
This method takes a list of pandas objects as arguments.

In [39]:
ds_full = pd.concat([ds_chars,states])
ds_full

,name,HP,home_state,state,nickname,capital
0,greg,200.0,WA,NaN,NaN,NaN
1,miles,200.0,WA,NaN,NaN,NaN
2,alan,170.0,TX,NaN,NaN,NaN
3,alison,300.0,DC,NaN,NaN,NaN
4,rachel,200.0,TX,NaN,NaN,NaN
0,NaN,NaN,NaN,WA,evergreen,Olympia
1,NaN,NaN,NaN,TX,alamo,Austin
2,NaN,NaN,NaN,DC,district,Washington
3,NaN,NaN,NaN,OH,buckeye,Columbus
4,NaN,NaN,NaN,OR,beaver,Salem


pd.concat()–– and many other pandas operations –– make use of an axis parameter. For this particular method I need to specify whether I want to concatenate the DataFrames row-wise (axis=0) or column-wise (axis=1). The default is axis=0, so let's override that!

In [40]:
ds_full = pd.concat([ds_chars,states],axis=1)
ds_full

,name,HP,home_state,state,nickname,capital
0,greg,200,WA,WA,evergreen,Olympia
1,miles,200,WA,TX,alamo,Austin
2,alan,170,TX,DC,district,Washington
3,alison,300,DC,OH,buckeye,Columbus
4,rachel,200,TX,OR,beaver,Salem


### EXTRA
### King County Assessments
As data scientists, we want to build a model to predict the sale price of a house in Seattle in 2019, based on its square footage. We know that the King County Department of Assessments has comprehensive data available on real property sales in the Seattle area. We need to prepare the data.

### First, get the data!
Go here and download two files: "Real Property Sales" and "Residential Building". Then unzip them. (Or you can run the cells below if you prefer.)

In [41]:
# %%bash
# cd data
# curl -o property_sales.zip https://aqua.kingcounty.gov/extranet/assessor/Real%20Property%20Sales.zip 

In [42]:
# %%bash
# cd data
# curl -o res_bldg.zip https://aqua.kingcounty.gov/extranet/assessor/Residential%20Building.zip 

In [43]:
# zf = ZipFile('data/property_sales.zip', 'r')
# zf.extractall('data')
# zf.close()

In [44]:
# zf = ZipFile('data/res_bldg.zip', 'r')
# zf.extractall('data')
# # zf.close()


In [45]:
# You'll need to use a new encoding here. List of all encodings here:
# https://docs.python.org/3/library/codecs.html#standard-encodings

# Both of these csv files have many columns, so we'll just pre-select
# which ones we want to use.

sales_df = pd.read_csv('data/EXTR_RPSale.csv',
                       encoding='latin-1',
                       usecols=['Major','Minor','DocumentDate','SalePrice']
                       )
sales_df

<ipython-input-45-8bcf2961f37b>:7: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  sales_df = pd.read_csv('data/EXTR_RPSale.csv',


,Major,Minor,DocumentDate,SalePrice
0,4000,228,04/29/1997,103500
1,0,0,06/05/1996,7000
2,160460,1802,12/14/2021,775000
3,282607,9118,08/20/1992,147000
4,534160,120,10/16/1995,102000
...,...,...,...,...
2340318,937630,1111,03/04/2009,355000
2340319,147165,30,07/14/1995,179000
2340320,751850,6330,08/15/1986,64000
2340321,178940,780,01/03/2006,0


In [46]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2340323 entries, 0 to 2340322
Data columns (total 4 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   Major         object
 1   Minor         object
 2   DocumentDate  object
 3   SalePrice     int64 
dtypes: int64(1), object(3)
memory usage: 71.4+ MB


In [47]:
bldg_df = pd.read_csv('data/EXTR_ResBldg.csv',
                      usecols=['Major','Minor','SqFtTotLiving','ZipCode']
                      )
bldg_df

<ipython-input-47-6904470f4655>:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  bldg_df = pd.read_csv('data/EXTR_ResBldg.csv',


,Major,Minor,ZipCode,SqFtTotLiving
0,6600,300,98126,760
1,6600,340,98136,1310
2,6600,345,98136,2040
3,6600,370,98126,1380
4,6600,390,98126,930
...,...,...,...,...
528283,6600,230,98136.0,1270
528284,6600,235,98136.0,2220
528285,6600,240,98136.0,1510
528286,6600,240,98136.0,960


In [48]:
bldg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 528288 entries, 0 to 528287
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Major          528288 non-null  int64 
 1   Minor          528288 non-null  int64 
 2   ZipCode        473712 non-null  object
 3   SqFtTotLiving  528288 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 16.1+ MB


In [49]:
sales_data = pd.merge(sales_df,bldg_df,on=['Major','Minor'])

In [50]:
sales_data.head()

,Major,Minor,DocumentDate,SalePrice,ZipCode,SqFtTotLiving
0,4000,228,04/29/1997,103500,98168,1560
1,4000,228,10/15/2014,221900,98168,1560
2,4000,228,08/28/2020,0,98168,1560
3,4000,228,05/06/2005,198000,98168,1560
4,4000,228,04/26/2019,369000,98168,1560


In [51]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1632486 entries, 0 to 1632485
Data columns (total 6 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Major          1632486 non-null  object
 1   Minor          1632486 non-null  object
 2   DocumentDate   1632486 non-null  object
 3   SalePrice      1632486 non-null  int64 
 4   ZipCode        1479426 non-null  object
 5   SqFtTotLiving  1632486 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 87.2+ MB


In [52]:
sales_data.loc[sales_data['ZipCode'].isna()].head()

,Major,Minor,DocumentDate,SalePrice,ZipCode,SqFtTotLiving
5,160460,1802,12/14/2021,775000,NaN,2160
6,160460,1802,11/15/2011,315000,NaN,2160
29,873300,95,08/20/2020,720000,NaN,2910
30,873300,95,06/11/2019,325000,NaN,2910
57,27200,130,05/19/1998,92000,NaN,2150


### Exercise
#### 1.What percentage of housing records are missing ZIP codes?

In [53]:
sales_data['ZipCode'].isna().value_counts(normalize=True) #9%

False    0.906241
True     0.093759
Name: ZipCode, dtype: float64

In [54]:
len(sales_data[sales_data['ZipCode'].isna()])/len(sales_data) #9%

0.09375884387369937

In [55]:
# Teacher code
sales_data['ZipCode'].isna().sum()/ sales_data.shape[0] #%9

0.09375884387369937

#### 2.Let's drop the rows with missing zip codes.

In [56]:
sales_data['ZipCode'].isna().sum()

153060

In [57]:
sales_data.shape[0]

1632486

In [58]:
sales_data.dropna(subset=['ZipCode'],inplace=True)
sales_data


,Major,Minor,DocumentDate,SalePrice,ZipCode,SqFtTotLiving
0,4000,228,04/29/1997,103500,98168,1560
1,4000,228,10/15/2014,221900,98168,1560
2,4000,228,08/28/2020,0,98168,1560
3,4000,228,05/06/2005,198000,98168,1560
4,4000,228,04/26/2019,369000,98168,1560
...,...,...,...,...,...,...
1632477,885670,200,02/19/1982,136373,98052.0,3180
1632479,172607,9002,05/17/1991,0,98019,3410
1632482,12605,9008,10/01/1993,110000,98072,790
1632483,279430,20,05/12/1997,281950,98007,2490


In [59]:
#Teacher code
sales_data = sales_data.loc[~sales_data['ZipCode'].isna(), :]
#sales_data2 = sales_data[~sales_data['ZipCode'].isna()]
sales_data

,Major,Minor,DocumentDate,SalePrice,ZipCode,SqFtTotLiving
0,4000,228,04/29/1997,103500,98168,1560
1,4000,228,10/15/2014,221900,98168,1560
2,4000,228,08/28/2020,0,98168,1560
3,4000,228,05/06/2005,198000,98168,1560
4,4000,228,04/26/2019,369000,98168,1560
...,...,...,...,...,...,...
1632477,885670,200,02/19/1982,136373,98052.0,3180
1632479,172607,9002,05/17/1991,0,98019,3410
1632482,12605,9008,10/01/1993,110000,98072,790
1632483,279430,20,05/12/1997,281950,98007,2490


### Time Permitting: Data Cleaning with Pandas
#### 1. Investigate and drop rows with invalid values in the SalePrice and SqFtTotLiving columns.

In [60]:
sales_data['SalePrice'].value_counts() #0 appears to be  a placeholder

0         511407
250000      4738
300000      4690
350000      4629
400000      4311
           ...  
19529          1
309227         1
109610         1
727250         1
136373         1
Name: SalePrice, Length: 76989, dtype: int64

In [61]:
sales_data['SqFtTotLiving'].value_counts() #everything seems okay here

1800     8888
1560     8762
1440     8735
1300     8671
1900     8384
         ... 
1171        1
16430       1
3437        1
3061        1
1473        1
Name: SqFtTotLiving, Length: 3643, dtype: int64

In [62]:
sales_data= sales_data[sales_data['SalePrice'] > 10000]
sales_data

,Major,Minor,DocumentDate,SalePrice,ZipCode,SqFtTotLiving
0,4000,228,04/29/1997,103500,98168,1560
1,4000,228,10/15/2014,221900,98168,1560
3,4000,228,05/06/2005,198000,98168,1560
4,4000,228,04/26/2019,369000,98168,1560
7,282607,9118,08/20/1992,147000,98014,1780
...,...,...,...,...,...,...
1632476,737960,40,10/28/1991,157000,98007,1750
1632477,885670,200,02/19/1982,136373,98052.0,3180
1632482,12605,9008,10/01/1993,110000,98072,790
1632483,279430,20,05/12/1997,281950,98007,2490


#### 2.2. Investigate and handle non-numeric ZipCode values
Can you find a way to shorten ZIP+4 codes to the first five digits?

In [63]:
#check non_numeric
import numpy as np
np.float = np.float64
x= sales_data[~sales_data['ZipCode'].apply(np.isreal)]
#x.to_excel('test.xlsx')

In [64]:
#sales_data['ZipCode'].map(lambda x: int(str(x)[:5]))

In [65]:
sales_data[sales_data['ZipCode']== '98028-3200']


,Major,Minor,DocumentDate,SalePrice,ZipCode,SqFtTotLiving
819031,402770,1219,04/08/2022,1600000,98028-3200,2770


In [66]:
example = sales_data['ZipCode'].iloc[819031]
int(str(example)[:5])

98038

In [67]:
### Teachers code
def is_integer(x):
    try:
        _ = int(x)
    except ValueError:
        return False
    return True

sales_data.loc[sales_data['ZipCode'].apply(is_integer) == False, 'ZipCode'].head()


12965    98199-3014
12966    98199-3014
12967    98199-3014
22632    98028-8908
37127    98042-3001
Name: ZipCode, dtype: object

In [68]:
#extension of teachers code
def five_digit_ZIP(x):
    try:
        return int(str(x)[:5])
    except:
        return x
sales_data['ZipCode'] = sales_data['ZipCode'].map(five_digit_ZIP)
sales_data = sales_data.loc[sales_data['ZipCode'].apply(is_integer) == True, :]
sales_data['ZipCode'] = sales_data['ZipCode'].map(int)

<ipython-input-68-14d3d44b04f3>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_data['ZipCode'] = sales_data['ZipCode'].map(five_digit_ZIP)
<ipython-input-68-14d3d44b04f3>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_data['ZipCode'] = sales_data['ZipCode'].map(int)


### 3. Add a column for PricePerSqFt

In [69]:
sales_data['ZipCode'].to_excel('test.xlsx')

In [70]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 962663 entries, 0 to 1632484
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Major          962663 non-null  object
 1   Minor          962663 non-null  object
 2   DocumentDate   962663 non-null  object
 3   SalePrice      962663 non-null  int64 
 4   ZipCode        962663 non-null  int64 
 5   SqFtTotLiving  962663 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 51.4+ MB


In [71]:
sales_data['PricePerSqFt'] = sales_data['SalePrice'] / sales_data['SqFtTotLiving']

<ipython-input-71-21166903640d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_data['PricePerSqFt'] = sales_data['SalePrice'] / sales_data['SqFtTotLiving']


In [72]:
sales_data.head()

,Major,Minor,DocumentDate,SalePrice,ZipCode,SqFtTotLiving,PricePerSqFt
0,4000,228,04/29/1997,103500,98168,1560,66.346154
1,4000,228,10/15/2014,221900,98168,1560,142.243590
3,4000,228,05/06/2005,198000,98168,1560,126.923077
4,4000,228,04/26/2019,369000,98168,1560,236.538462
7,282607,9118,08/20/1992,147000,98014,1780,82.584270


### 4. Subset the data to 2021 sales only.
We can assume that the DocumentDate is approximately the sale date.

In [73]:
sales_data['Year'] = pd.to_datetime(sales_data['DocumentDate'],format='%m/%d/%Y').dt.year

<ipython-input-73-638b6d124334>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_data['Year'] = pd.to_datetime(sales_data['DocumentDate'],format='%m/%d/%Y').dt.year


In [74]:
sales_data.head()

,Major,Minor,DocumentDate,SalePrice,ZipCode,SqFtTotLiving,PricePerSqFt,Year
0,4000,228,04/29/1997,103500,98168,1560,66.346154,1997
1,4000,228,10/15/2014,221900,98168,1560,142.243590,2014
3,4000,228,05/06/2005,198000,98168,1560,126.923077,2005
4,4000,228,04/26/2019,369000,98168,1560,236.538462,2019
7,282607,9118,08/20/1992,147000,98014,1780,82.584270,1992


In [75]:
sales_data1 =sales_data[sales_data['Year'] == 2021] #assign to variable sales_data to change the dataset to have 2021 values only
#in my case used a diffrent table name but shld use sales_date
sales_data1

,Major,Minor,DocumentDate,SalePrice,ZipCode,SqFtTotLiving,PricePerSqFt,Year
10,282607,9118,10/11/2021,516720,98014,1780,290.292135,2021
375,287300,690,06/11/2021,680000,98031,2130,319.248826,2021
379,756460,230,09/07/2021,715000,98057,1880,380.319149,2021
438,118000,275,09/29/2021,311000,98178,2880,107.986111,2021
459,638600,85,06/18/2021,1075000,98115,1810,593.922652,2021
...,...,...,...,...,...,...,...,...
1632270,276760,3054,06/24/2021,985000,98107,1530,643.790850,2021
1632274,701610,500,10/28/2021,950000,98011,1160,818.965517,2021
1632276,177760,170,12/16/2021,1500000,98006,2930,511.945392,2021
1632374,524480,1455,08/04/2021,1400000,98109,2560,546.875000,2021


In [76]:
#Teachers code, doesnt make sense since the data has 2023 as well
df= sales_data.copy()
df['DocumentDate'] = pd.to_datetime(df['DocumentDate'])
df = df.loc[df['DocumentDate'] > '12/31/2020']
df


,Major,Minor,DocumentDate,SalePrice,ZipCode,SqFtTotLiving,PricePerSqFt,Year
10,282607,9118,2021-10-11,516720,98014,1780,290.292135,2021
53,510454,400,2023-08-15,1044000,98038,3230,323.219814,2023
67,807840,240,2023-05-31,1300000,98074,1730,751.445087,2023
84,507140,104,2022-09-27,1395000,98115,2720,512.867647,2022
88,62205,9001,2024-05-10,1220000,98055,1820,670.329670,2024
...,...,...,...,...,...,...,...,...
1632360,408330,5648,2023-07-07,935000,98103,1360,687.500000,2023
1632374,524480,1455,2021-08-04,1400000,98109,2560,546.875000,2021
1632425,894510,220,2021-03-12,436000,98023,1730,252.023121,2021
1632474,955780,100,2022-04-29,500000,98148,1200,416.666667,2022


In [77]:
#to add to teacher code
df= sales_data.copy()
df['DocumentDate'] = pd.to_datetime(df['DocumentDate'])
df = df.loc[(df['DocumentDate'] > '12/31/2020') & (df['DocumentDate'] < '01/01/2022')]
df

,Major,Minor,DocumentDate,SalePrice,ZipCode,SqFtTotLiving,PricePerSqFt,Year
10,282607,9118,2021-10-11,516720,98014,1780,290.292135,2021
375,287300,690,2021-06-11,680000,98031,2130,319.248826,2021
379,756460,230,2021-09-07,715000,98057,1880,380.319149,2021
438,118000,275,2021-09-29,311000,98178,2880,107.986111,2021
459,638600,85,2021-06-18,1075000,98115,1810,593.922652,2021
...,...,...,...,...,...,...,...,...
1632270,276760,3054,2021-06-24,985000,98107,1530,643.790850,2021
1632274,701610,500,2021-10-28,950000,98011,1160,818.965517,2021
1632276,177760,170,2021-12-16,1500000,98006,2930,511.945392,2021
1632374,524480,1455,2021-08-04,1400000,98109,2560,546.875000,2021


### 5. What is the mean price per square foot for a house sold in Seattle in 2021?

In [78]:
sales_data1['PricePerSqFt'].mean()

506.2470343438183

### Level Up: pandas.set_option()
We can adjust how pandas works by setting options in advance.

### Block Scientific Notation
For example, suppose we want to prevent numbers from being displayed in scientific notation.

In [79]:
df = pd.DataFrame([[1e9,2e9],[3e9,4e9]])
df

,0,1
0,1.000000e+09,2.000000e+09
1,3.000000e+09,4.000000e+09


Then we can use:

In [80]:
pd.set_option('display.float_format','{:.2f}'.format)
df

,0,1
0,1000000000.00,2000000000.00
1,3000000000.00,4000000000.00


### See More Rows
Or suppose we want pandas to show more rows.

In [81]:
df2 = pd.DataFrame(np.array(range(100)))
df2

,0
0,0
1,1
2,2
3,3
4,4
...,...
95,95
96,96
97,97
98,98


In [82]:
pd.set_option('display.max_rows',100)

In [83]:
df2

,0
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9
